In [1]:
from directory_cobveg_treat import definir_diretorio
project_path = definir_diretorio()

import os
import geopandas as gpd

from utils import (
    get_data_diretorio,
    save_parquet_excel,
)

# Porcentagem do distrito com cobertura vegetal

In [2]:
data_path = get_data_diretorio(project_path)

# Dependências

Este notebook é dependente dos assets resultante dos notebooks 'overlay_distrito_cobertura_vegetal' e '../../arborizacao_viaria/carregar_dados/malha_distritos'

In [3]:
overlay_path = os.path.join(
    data_path, 
    'assets',
    'cobertura_vegetal',
    'overlay_cobertura_vegetal_distritos.parquet'
)
overlay_gdf = gpd.read_parquet(overlay_path)

In [4]:
overlay_gdf.sample(2)

,cd_cob_veg,cd_identi0,cd_categor,cd_subcate,cd_subcat0,qt_altura_,area_cob_veg,CD_REGIAO,NM_REGIAO,CD_UF,...,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_recort,perc_recort
13608,13518.0,13550.0,11.0,0.0,0.0,0.0,143.442482,3,Sudeste,35,...,350001,São Paulo,3550308,São Paulo/SP,4.533105,24730,10783,"POLYGON ((321223.417 7398957.164, 321222.755 7...",143.442482,1.0
38640,38190.0,38316.0,14.0,0.0,0.0,0.0,141.516247,3,Sudeste,35,...,350001,São Paulo,3550308,São Paulo/SP,7.498376,92186,40616,"POLYGON ((341509.508 7393649.366, 341510.545 7...",141.516247,1.0


In [5]:
distritos_path = os.path.join(data_path, 'assets', 'distrito_ibge.parquet')
gdf_distritos = gpd.read_parquet(distritos_path)

# Merge dos gdfs

In [6]:
gdf_dist_cobveg = gdf_distritos.merge(
    overlay_gdf[[
        'cd_cob_veg',
        'area_recort',
        'perc_recort',
        'CD_DIST'
    ]],
    on='CD_DIST'
)

In [7]:
gdf_dist_cobveg.shape

(274601, 21)

In [8]:
gdf_distritos.shape

(96, 18)

# Calcular área de cobertura vegetal por distrito

In [9]:
for distrito in gdf_distritos['CD_DIST']:
    area_cobveg = sum(
            gdf_dist_cobveg['area_recort']
            .loc[gdf_dist_cobveg['CD_DIST']==distrito]
    )
    
    (gdf_distritos
        .loc[
            gdf_distritos['CD_DIST']==distrito, 
            'area_dist_cobveg'
        ])= area_cobveg

In [10]:
gdf_distritos.sample(2)

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_dist_cobveg
80,3,Sudeste,35,São Paulo,3550308,São Paulo,355030881,TREMEMBE,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,57.709814,196563,78219,"POLYGON ((338977.39 7414197.122, 338983.98 741...",4.264705e+07
39,3,Sudeste,35,São Paulo,3550308,São Paulo,355030840,JAGUARA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,4.533105,24730,10783,"POLYGON ((320979.577 7398155.785, 320973.947 7...",9.650457e+05


# Cálculo da porcentagem da área

Antes de tudo, vou confirmar se a coluna 'AREA_KM_2' pode ser usada como base para o cálculo

In [11]:
x=(
    gdf_distritos['geometry']
    .area
    .loc[gdf_distritos['CD_DIST']=='355030891']
)
print(x.round(3))

90    8896709.007
dtype: float64


In [12]:
print(
    gdf_distritos['AREA_KM2']
    .loc[gdf_distritos['CD_DIST']=='355030891']
)

90    8.898548
Name: AREA_KM2, dtype: float64


Aparentemente a área proveniente do cálculo da geometria não é em Km², então faremos uma nova coluna para calcular a área do distrito segundo a geometria

In [13]:
gdf_distritos['area_distrito'] = gdf_distritos['geometry'].area


In [14]:
gdf_distritos.sample(2)

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_dist_cobveg,area_distrito
53,3,Sudeste,35,São Paulo,3550308,São Paulo,355030854,MORUMBI,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,11.462372,43690,20710,"POLYGON ((325192.046 7387331.143, 325166.217 7...",5.102053e+06,1.146180e+07
64,3,Sudeste,35,São Paulo,3550308,São Paulo,355030865,RAPOSO TAVARES,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.164313,117738,49006,"POLYGON ((319022.919 7388900.499, 318990.285 7...",5.316876e+06,1.216456e+07


In [15]:
gdf_distritos['pct_dist_cobveg'] = (
    gdf_distritos['area_dist_cobveg'] / gdf_distritos['area_distrito']
)

In [16]:
gdf_distritos.sample(2)

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,...,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_dist_cobveg,area_distrito,pct_dist_cobveg
66,3,Sudeste,35,São Paulo,3550308,São Paulo,355030867,RIO PEQUENO,3501,São Paulo,...,São Paulo,3550308,São Paulo/SP,9.755869,131631,54686,"POLYGON ((322346.814 7391673.431, 322303.81 73...",2.593782e+06,9.755831e+06,0.265870
57,3,Sudeste,35,São Paulo,3550308,São Paulo,355030858,PEDREIRA,3501,São Paulo,...,São Paulo,3550308,São Paulo/SP,18.483827,163586,64002,"POLYGON ((333773.279 7378335.002, 333775.099 7...",6.284694e+06,1.848196e+07,0.340045


# Conferir compatibilidade e consistência

Agora vamos procurar por alguma inconsistência no cálculo da porcentagem. Caso algum valor de 'pct_dist_cobveg' seja maior ou igual a 1, seria como se a totalidade da área do distrito, ou mais do que ela, é de cobertura vegetal

In [17]:
gdf_distritos[gdf_distritos['pct_dist_cobveg']>=1]

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,...,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_dist_cobveg,area_distrito,pct_dist_cobveg


In [18]:
sum(gdf_distritos['pct_dist_cobveg'].isnull())

0

In [27]:
max_pct=gdf_distritos['pct_dist_cobveg'].max()
max_name=gdf_distritos['NM_DIST'].loc[gdf_distritos['pct_dist_cobveg']==max_pct].iloc[0]
print(f'{max_name} é o distrito com a maior porcentagem de cobertura vegetal: {round(max_pct*100, 2)}%')

MARSILAC é o distrito com a maior porcentagem de cobertura vegetal: 97.12%


In [29]:
min_pct=gdf_distritos['pct_dist_cobveg'].min()
min_name = gdf_distritos['NM_DIST'].loc[gdf_distritos['pct_dist_cobveg']==min_pct].iloc[0]
print(f'{min_name} é o distrito com a menor porcentagem de cobertura vegetal: {round(min_pct*100, 2)}%')

BRAS é o distrito com a menor porcentagem de cobertura vegetal: 5.57%


# Porcentagem do Município de São Paulo

In [22]:
gdf_distritos.sample(2)

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,...,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_dist_cobveg,area_distrito,pct_dist_cobveg
7,3,Sudeste,35,São Paulo,3550308,São Paulo,355030808,BELEM,3501,São Paulo,...,São Paulo,3550308,São Paulo/SP,6.121560,55785,25449,"POLYGON ((337329.398 7395388.459, 337268.595 7...",8.928170e+05,6.120671e+06,0.145869
42,3,Sudeste,35,São Paulo,3550308,São Paulo,355030843,JARDIM ANGELA,3501,São Paulo,...,São Paulo,3550308,São Paulo/SP,36.738281,311432,125669,"POLYGON ((318894.224 7372253.527, 318886.412 7...",1.504344e+07,3.673837e+07,0.409475


In [25]:
area_sp = sum(gdf_distritos['area_distrito'])
area_cobveg_total = sum(gdf_distritos['area_dist_cobveg'])
pct_sp = area_cobveg_total / area_sp
print(f"{round(pct_sp*100, 2)}% do município de São Paulo é área de cobertura vegetal")

47.96% do município de São Paulo é área de cobertura vegetal


# Salvar gdf

In [21]:
save_parquet_excel(
    gdf_distritos, 
    fname= 'distritos_cobertura_vegetal',
    data_path = data_path,
    data_subpath = os.path.join(
        'assets',
        'cobertura_vegetal'
    )
)